In [9]:
# !pip3 install mljar-supervised

In [25]:
from supervised.automl import AutoML # mljar-supervised
import pandas as pd
# import pdpipe as pdp

In [26]:
# full_players = pd.read_csv('full_players.csv', index_col=0).reset_index(drop=True)

# # Opponent features
# full_players['']

In [27]:
Players = pd.read_csv('full_players.csv',index_col=0).dropna()
Players['Score'] = pd.to_numeric(Players['Score'], errors='coerce')
Players2020 = Players[Players['Year']==2020]
TargerScores = Players2020['Score']
Players2019 = Players[Players['Year']==2019]

In [28]:
Players2020 = Players2020[['Team 1', 'Name', 'Position', 'Price', 'Score']].drop_duplicates().rename(columns = {'Score' : 'Target_score'})

Players2019 = Players2019[['Team 1', 'Team 2', 'Name', 'Position', 'Score', ]].drop_duplicates()

In [29]:
Players = Players2019.merge(Players2020, on=['Team 1', 'Name', 'Position'])
"""
Openent Metrics: Averages for people who played against same oponents
    OM_FO_AVG_SCORE
    OM_MI_AVG_SCORE
    OM_RU_AVG_SCORE
    OM_DE_AVG_SCORE
Player metrics
    PM_AVG_SCORE
    PM_AVG_NON_ZERO_SCORE
"""

# Players['OM_FO_AVG_SCORE'] = 
Opponent = Players[['Team 1', 'Position', 'Score','Price']].groupby(['Team 1', 'Position']).mean().reset_index()
Opponent = Opponent.rename(columns={'Team 1': 'Opponent', 'Position':'Oponent Relevant Position'})

pd.melt(Opponent, id_vars=["Opponent",'Score','Price'], value_vars=[ 'Oponent Relevant Position'], )
Opponent = Opponent.pivot(index=['Opponent'], columns=['Oponent Relevant Position']).reset_index()
Opponent.columns = ['OponentAVG_'+col[0]+'_'+col[1] for col in Opponent.columns]
features = list(Opponent.columns)
features.remove('OponentAVG_Opponent_')
Players = Players.merge(Opponent, left_on = ['Team 2'], right_on=['OponentAVG_Opponent_'])
Players = Players[Players['Position']=='MI']
Players['Players_Average_Score'] = Players[['Name','Team 1','Score']].groupby(['Name','Team 1']).transform('mean')['Score']
features += ['Players_Average_Score']

In [30]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from supervised.automl import AutoML # mljar-supervised

# Load the data
X_train, X_test, y_train, y_test = train_test_split(
    Players[features],
    Players['Score'],
    test_size=0.25,
    random_state=123,
)

# train models with AutoML
automl = AutoML(mode="Explain")
automl.fit(X_train, y_train)

# compute the MSE on test data
predictions = automl.predict(X_test)
print("Test MSE:", mean_squared_error(y_test, predictions))

Linear algorithm was disabled.
AutoML directory: AutoML_1
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Baseline', 'Decision Tree', 'Random Forest', 'Xgboost', 'Neural Network']
AutoML will ensemble availabe models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 2 models
1_Baseline rmse 26.674174 trained in 0.36 seconds
2_DecisionTree rmse 20.869702 trained in 15.36 seconds
* Step default_algorithms will try to check up to 3 models
3_Default_Xgboost rmse 5.59852 trained in 58.19 seconds
4_Default_NeuralNetwork rmse 20.361794 trained in 2.87 seconds
5_Default_RandomForest rmse 20.378229 trained in 7.48 seconds
* Step ensemble will try to check up to 1 model
Ensemble rmse 5.59852 trained in 0.09 seconds
AutoML fit time: 90.16 seconds
AutoML best model: 3_Default_Xgboost


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').